## Adapt section 5, *Learning to balance*, using the Cart Pole environment of Gymnasium

### Main steps to take
- gather samples of the system's dynamics by running a random policy
- learn a local linear model of the system's dynamic around the point $\theta = 0$
- modify the reward function in equation (4) to reflect that in our problem the acceleration input can take only 2 values
- compute a balancing policy by leveraging the local linear model and reward function (e.g. LQR controller)

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd

In [ ]:
env = gym.make("CartPole-v1")
# an observation is a tuple (Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity)
print(env.observation_space)
# there are two possible actions, 0 and 1, which correspond to pushing the cart left or right
print(env.action_space)

In [ ]:
def generate_sample_of_system_dynamics(env, nb_samples):
  """ 
  Returns: a dataframe containing predictors and outcomes
  """
  
  samples = []
  s, _ = env.reset()
  for _ in range(nb_samples):
    a = np.random.randint(0, 2)
    next_s, _, d, _, _ = env.step(a)
    samples.append({
      'x': s[0], 
      'x_dot': s[1], 
      'theta': s[2], 
      'theta_dot': s[3], 
      '2u - 1': 2 * a - 1, # mapping inputs to {-1, 1} rather than {0, 1} is important, because otherwise the linear regression needs a non-zero constant to fit well
      'evolution_x': next_s[0], 
      'evolution_x_dot': next_s[1], 
      'evolution_theta_dot': next_s[3]})
    if d:
      s, _ = env.reset()
    else:
      s = next_s
  return pd.DataFrame(samples)

In [ ]:
env = gym.make("CartPole-v1")
samples = generate_sample_of_system_dynamics(env, 10000)
print(samples.describe())

## Fitting a linear model for the system's dynamics around $\theta=0$

Under the assumption thata the pole's angular velocity is a linear
function of the state and action we use linear regression to find
the coefficients based on sample data

In [ ]:
from statsmodels.api import OLS

small_angle_samples = samples[np.abs(samples["theta"]) < 0.05]
predictors = small_angle_samples[['x', 'x_dot', 'theta', 'theta_dot', '2u - 1']]
outcomes = ['evolution_x_dot', 'evolution_theta_dot', 'evolution_x']
for o in outcomes:
  model = OLS(small_angle_samples[o], predictors)
  fit = model.fit()
  print(f"{o} linear model:")
  print(fit.params)
  print(f"Mean squared error of {o} fit", fit.mse_resid)

## Computing a state-feedback controller according to a discrete-time linear quadratic regulator design

In [ ]:
from control import dlqr

dT = 0.02
A = np.array([
  [1, dT, 0, 0], #x
  [0, 1, -0.014107, 0], #x_dot
  [0, 0, 1, dT], #theta
  [0.000031, 0.000019, 0.315099, 0.999998] #theta_dot
])
B = np.array([
  [0], #x
  [0.195111], #x_dot
  [0], #theta
  [-0.292548] #theta_dot
])
Q = np.diag([125, 50, 1200, 25])
R = np.diag([1.5])

K, S, E = dlqr(A, B, Q, R)
print(K)

## Trying out the learned control policy

In [ ]:
env = gym.make("CartPole-v1", render_mode='human')
s, _ = env.reset()
while True:
  print(np.dot(-K, s))
  a = 0 if np.dot(-K, s) < 0.5 else 1
  s, _, dead, trunc, _ = env.step(a)
  if dead or trunc:
    break

In [ ]:
env.close()